In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('./eval.txt', 'r') as f:
    results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t') for r in results if '\t' in r]
results_dict = {}
for r in results: # S
    model_name, dataset_name, metric_name, metric_value = r
    key = dataset_name
    subkey = metric_name
    subsubkey = model_name
    values = list(set(r) - set([key, subkey, subsubkey]))
    subsubkey = '-'.join(subsubkey.split('-')[2:])
    if key not in results_dict:
        results_dict[key] = {} # .append(values)
    # else:
    #     results_dict[key] = {} #[values]
    if subkey not in results_dict[key]:
        results_dict[key][subkey] = {}
    if subsubkey in results_dict[key][subkey]:
        results_dict[key][subkey][subsubkey] += values
    else:
        results_dict[key][subkey][subsubkey] = values

# results_dict

In [3]:
def remove_underscore(val): # Remove underscore, and keep the part after the underscore
    return val.split('_')[-1]
def map2d(func, grid): # Mapping for 2d arrays, from: https://stackoverflow.com/questions/70742445/elegant-map-over-2d-list
    return [[func(value) for value in row] for row in grid]

df = pd.DataFrame(results, columns = ['model', 'dataset', 'metric', 'value'])
df['value'] = pd.to_numeric(df['value'])
df['model'] = df['model'].map(lambda x: '-'.join(x.split('-')[2:]))
model_names_list = df['model'].str.split('-').tolist()
# Remove all underscores from our 2d list
model_names_list = map2d(remove_underscore, model_names_list)
model_names = np.array(model_names_list)

# Splitting model name into more columns
df['train_data'] = np.char.add(model_names[:, 0], model_names[:, 1])
df['ratio'] = model_names[:, 2]
df['vision_model'] = model_names[:, 3]
df['method'] = model_names[:, 4]
df['keyword'] = model_names[:, 5]
df['AL'] = model_names[:, 6]
df['PL'] = model_names[:, 7]
df['vit_AL'] = model_names[:, 8]
df.drop('model', axis = 1, inplace = True)

cols = sorted(df.columns.tolist())
# The ratio have the format: 'ratio_xxx' where xxx is some float.
# We only want that float value, and need to convert it from string to float.
df['ratio'] = pd.to_numeric(df['ratio'])

# Group by all columns except the performance values, then compute mean, std and number of model runs for the performance
df = df.groupby(list(set(cols)-set(['value']))).agg({'value':['mean', 'std', 'count']})

In [4]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

value  \
                                                                                                          mean   
PL         ratio train_data vit_AL AL    method dataset   metric             vision_model keyword                
None       0.1   RSALL      False  False base   RSICD     image_to_text_R@1  RN50         none        4.658000   
                                                          image_to_text_R@10 RN50         none       29.726000   
                                                          image_to_text_R@5  RN50         none       17.660000   
                                                          text_to_image_R@1  RN50         none        4.370000   
                                                          text_to_image_R@10 RN50         none       28.664000   
                                                          text_to_image_R@5  RN50         none       16.782000   
                                                RSICD-CLS zeroshot-val-top1  RN50         none       59.746000   
                                                Sydney    image_to_text_R@1  RN50         none        3.422000   
                                                          image_to_text_R@10 RN50         none       19.074000   
                                                          image_to_text_R@5  RN50         none       11.790000   
                                                          text_to_image_R@1  RN50         none        4.710000   
                                                          text_to_image_R@10 RN50         none       23.624000   
                                                          text_to_image_R@5  RN50         none       14.568000   
                                                UCM       image_to_text_R@1  RN50         none        3.298000   
                                                          image_to_text_R@10 RN50         none       21.596000   
                                                          image_to_text_R@5  RN50         none       13.058000   
                                                          text_to_image_R@1  RN50         none        3.880000   
                                                          text_to_image_R@10 RN50         none       21.344000   
                                                          text_to_image_R@5  RN50         none       13.084000   
                                                UCM-CLS   zeroshot-val-top1  RN50         none       67.610000   
                                   True  base   RSICD     image_to_text_R@1  RN50         none        3.110000   
                                                          image_to_text_R@10 RN50         none       20.293333   
                                                          image_to_text_R@5  RN50         none       11.701667   
                                                          text_to_image_R@1  RN50         none        3.108333   
                                                          text_to_image_R@10 RN50         none       20.418333   
                                                          text_to_image_R@5  RN50         none       11.958333   
                                                RSICD-CLS zeroshot-val-top1  RN50         none       39.060000   
                                                Sydney    image_to_text_R@1  RN50         none        2.045000   
                                                          image_to_text_R@10 RN50         none       13.583333   
                                                          image_to_text_R@5  RN50         none        7.410000   
                                                          text_to_image_R@1  RN50         none        2.346667   
                                                          text_to_image_R@10 RN50         none       15.561667   
                                                          text_to_image_R@5  RN50         none        9.118333   
 

In [5]:
df.to_csv('eval.csv')

In [8]:
df_2 = df.reset_index()
df_2 = df_2[(df_2['method'] == 'ours') & (df_2['ratio'] == 0.2) & (df_2['PL'] == 'ot.image')] # (df_2['metric'] == 'image_to_text_R@5') &
df_2 = df_2.sort_values(['metric', 'dataset'])
display(df_2)

PL ratio train_data vit_AL     AL method    dataset  \
                                                                 
120  ot.image   0.2      RSALL   True  False   ours      RSICD   
127  ot.image   0.2      RSALL   True  False   ours     Sydney   
133  ot.image   0.2      RSALL   True  False   ours        UCM   
121  ot.image   0.2      RSALL   True  False   ours      RSICD   
128  ot.image   0.2      RSALL   True  False   ours     Sydney   
134  ot.image   0.2      RSALL   True  False   ours        UCM   
122  ot.image   0.2      RSALL   True  False   ours      RSICD   
129  ot.image   0.2      RSALL   True  False   ours     Sydney   
135  ot.image   0.2      RSALL   True  False   ours        UCM   
123  ot.image   0.2      RSALL   True  False   ours      RSICD   
130  ot.image   0.2      RSALL   True  False   ours     Sydney   
136  ot.image   0.2      RSALL   True  False   ours        UCM   
124  ot.image   0.2      RSALL   True  False   ours      RSICD   
131  ot.image   0.2      RSALL   True  False   ours     Sydney   
137  ot.image   0.2      RSALL   True  False   ours        UCM   
125  ot.image   0.2      RSALL   True  False   ours      RSICD   
132  ot.image   0.2      RSALL   True  False   ours     Sydney   
138  ot.image   0.2      RSALL   True  False   ours        UCM   
126  ot.image   0.2      RSALL   True  False   ours  RSICD-CLS   
139  ot.image   0.2      RSALL   True  False   ours    UCM-CLS   

                 metric vision_model    keyword  value            
                                                  mean std count  
120   image_to_text_R@1         RN50  classname   0.55 NaN     1  
127   image_to_text_R@1         RN50  classname   1.21 NaN     1  
133   image_to_text_R@1         RN50  classname   0.24 NaN     1  
121  image_to_text_R@10         RN50  classname   3.56 NaN     1  
128  image_to_text_R@10         RN50  classname   7.04 NaN     1  
134  image_to_text_R@10         RN50  classname   2.38 NaN     1  
122   image_to_text_R@5         RN50  classname   1.92 NaN     1  
129   image_to_text_R@5         RN50  classname   4.23 NaN     1  
135   image_to_text_R@5         RN50  classname   1.37 NaN     1  
123   text_to_image_R@1         RN50  classname   0.37 NaN     1  
130   text_to_image_R@1         RN50  classname   1.41 NaN     1  
136   text_to_image_R@1         RN50  classname   0.60 NaN     1  
124  text_to_image_R@10         RN50  classname   3.93 NaN     1  
131  text_to_image_R@10         RN50  classname   9.86 NaN     1  
137  text_to_image_R@10         RN50  classname   4.05 NaN     1  
125   text_to_image_R@5         RN50  classname   1.83 NaN     1  
132   text_to_image_R@5         RN50  classname   5.23 NaN     1  
138   text_to_image_R@5         RN50  classname   2.20 NaN     1  
126   zeroshot-val-top1         RN50  classname  10.42 NaN     1  
139   zeroshot-val-top1         RN50  classname  17.95 NaN     1